In [1]:
import pandas as pd
import numpy as np

In [2]:
monday_df    = pd.read_csv('./data/monday.csv', sep=';', parse_dates=True, index_col=0)
tuesday_df   = pd.read_csv('./data/tuesday.csv', sep=';', parse_dates=True, index_col=0)
wednesday_df = pd.read_csv('./data/wednesday.csv', sep=';', parse_dates=True, index_col=0)
thusday_df   = pd.read_csv('./data/thursday.csv', sep=';', parse_dates=True, index_col=0)
friday_df    = pd.read_csv('./data/friday.csv', sep=';', parse_dates=True, index_col=0)
monday    = monday_df.copy()
tuesday   = tuesday_df.copy()
wednesday = wednesday_df.copy()
thursday  = thusday_df.copy()
friday    = friday_df.copy()
monday   ['day'] ='monday'
tuesday  ['day'] ='tuesday'
wednesday['day'] ='wednesday'
thursday ['day'] ='thursday'
friday   ['day'] ='friday'

days=[monday]#, tuesday, wednesday, thursday, friday]

In [3]:
monday_df.head(1)

,customer_no,location
timestamp,,
2019-09-02 07:03:00,1,dairy


In [4]:
def time_step(df):
    df=df.groupby('customer_no').resample('1Min').ffill()
    df = df.drop(columns=['customer_no'])
    df.reset_index(inplace=True)
    return df

In [5]:
def kickout(df):
    for idk in range(1, df['customer_no'].max()+1):
        if df.loc[df['customer_no']==idk, 'from'].iloc[-1] != 'checkout':
            #print(idk)
            temp_df = df.loc[df['customer_no']==idk].iloc[-1].to_frame().T
            temp_df.loc[temp_df['customer_no'] == idk,'from']     = temp_df.loc[temp_df['customer_no'] == idk,'to']
            temp_df.loc[temp_df['customer_no'] == idk,'to'] = 'checkout'
            #temp_df.loc[temp_df['customer_no'] == idk,'timestamp'] = temp_df.loc[temp_df['customer_no'] == idk,'timestamp'] + pd.DataOffset(minute=1)
            df=pd.concat([df, temp_df], ignore_index=True)
            #miss_m1_DATE = missing_val['DATE'] - pd.DateOffset(years=1

In [6]:
def to_from(df):
    df['from'] = df['location']
    df['to']   = df['location'].shift(1)
    df.drop(columns='location', inplace=True)
    df.fillna(value='checkout', inplace=True)
    return df

In [7]:
tm_list=[]
def get_tm(df):
    tm = pd.crosstab(monday['to'], monday['from'], normalize='index')
    tm.loc['entry'] = [1, 0, 0, 0, 0]
    return tm

In [8]:
for i, day in enumerate(days):
    days[i]    = time_step(day)
    days[i]    = to_from(day)
    print(day.head(1))
    #days[i]    = kickout(day)
    #days[i]    = time_step(day)
    #tm_list.append(get_tm(day))

                     customer_no     day   from        to
timestamp                                                
2019-09-02 07:03:00            1  monday  dairy  checkout


In [9]:
monday

,customer_no,day,from,to
timestamp,,,,
2019-09-02 07:03:00,1,monday,dairy,checkout
2019-09-02 07:03:00,2,monday,dairy,dairy
2019-09-02 07:04:00,3,monday,dairy,dairy
2019-09-02 07:04:00,4,monday,dairy,dairy
2019-09-02 07:04:00,5,monday,spices,dairy
...,...,...,...,...
2019-09-02 21:49:00,1442,monday,checkout,spices
2019-09-02 21:49:00,1444,monday,checkout,checkout
2019-09-02 21:49:00,1445,monday,dairy,checkout
